Importeren van de gebruikte modules:

In [9]:
import pandas as pd
from matplotlib import pyplot as plt

<h2>Data inlezen + prepareren</h2>

Inlezen van een CSV bestand. Hierbij moeten we opletten dat het scheidingsteken geen komma (,) is maar een puntkomma (;):

In [ ]:
# pd.read_csv('Bevolking_2021.csv')
inleesbestand = pd.read_csv('Bevolking_2021.csv', sep=';')
inleesbestand.head()

Splitsen van de mannen in een aparte dataframe. Hernoemen van de kolom 'mannen' in 'aantal' + kollom toevoegen van geslacht zodat deze kan samengevoegd worden met de vrouwen dataframe.

In [5]:
mannen = inleesbestand.drop(['Totaal','Vrouwen'], axis=1)
mannen.rename(columns={'Mannen':'Aantal'}, inplace=True)
mannen['Geslacht'] = "Mannen"
mannen.head()

,NIS code,Woonplaats,Aantal,Geslacht
0,1000,België,5677211,Mannen
1,2000,Vlaams Gewest,3293617,Mannen
2,3000,Waals Gewest,1784520,Mannen
3,4000,Brussels Hoofdstedelijk Gewest,599074,Mannen
4,10000,Provincie Antwerpen,931025,Mannen


Splitsen van de vrouwen in een aparte dataframe. Hernoemen van de kolom 'vrouwen' in 'aantal' + kollom toevoegen van geslacht zodat deze kan samengevoegd worden met de mannen dataframe.

In [6]:
vrouwen = inleesbestand.drop(['Totaal','Mannen'], axis=1)
vrouwen.rename(columns={'Vrouwen':'Aantal'}, inplace=True)
vrouwen['Geslacht'] = "Vrouwen"
vrouwen.head()

,NIS code,Woonplaats,Aantal,Geslacht
0,1000,België,5844027,Vrouwen
1,2000,Vlaams Gewest,3359445,Vrouwen
2,3000,Waals Gewest,1863686,Vrouwen
3,4000,Brussels Hoofdstedelijk Gewest,620896,Vrouwen
4,10000,Provincie Antwerpen,944499,Vrouwen


Samenvoegen van mannen en vrouwen dataframes:

In [ ]:
bevolking_2021 = pd.concat([mannen, vrouwen], ignore_index=True)

Aanduiding van het jaar toevoegen aan de dataset:

In [ ]:
bevolking_2021["Jaartal"] = 2021
bevolking_2021.head()

<h2>Data inlezen + prepareren (Automatisatie)</h2>

Maak een functie die als argument het jaartal neemt en voor het opgegeven jaartal alle bovenstaande handeling en het resultaat (als dataframe) retourneerd:

In [12]:
def inlezen_bevolking(jaartal):
    inleesbestand = pd.read_csv('Bevolking_' + str(jaartal) + '.csv', sep=';')
    
    mannen = inleesbestand.drop(['Totaal','Vrouwen'], axis=1)
    mannen.rename(columns={'Mannen':'Aantal'}, inplace=True)
    mannen['Geslacht'] = "Mannen"

    vrouwen = inleesbestand.drop(['Totaal','Mannen'], axis=1)
    vrouwen.rename(columns={'Vrouwen':'Aantal'}, inplace=True)
    vrouwen['Geslacht'] = "Vrouwen"

    bevolking = pd.concat([mannen, vrouwen], ignore_index=True)

    bevolking["Jaartal"] = jaartal
    
    return bevolking

Samenvoegen van alle datasets van de verschillende jaren in één dataset. Maak gebruik van de functie die je geschreven hebt:

In [13]:
bevolking = pd.concat([inlezen_bevolking(2021), inlezen_bevolking(2022), inlezen_bevolking(2023)], ignore_index=True)

# Test om te kijken of inlezen gelukt is
bevolking[bevolking['NIS code'] == 1000]

,NIS code,Woonplaats,Aantal,Geslacht,Jaartal
0,1000,België,5677211,Mannen,2021
638,1000,België,5844027,Vrouwen,2021
1276,1000,België,5708902,Mannen,2022
1914,1000,België,5875106,Vrouwen,2022
2552,1000,België,5761410,Mannen,2023
3190,1000,België,5936147,Vrouwen,2023


<h2>Data verrijken</h2>

Filter alle aggregatielijnen uit de tabel (België aggregatie lijn, alle 'Gewest' lijnen, alle 'Provincie' lijnen en alle 'Arrondissement' lijnen:

In [14]:
bevolking = bevolking[bevolking['Woonplaats'].str.find('België') == -1]
bevolking = bevolking[bevolking['Woonplaats'].str.find('Gewest') == -1]
bevolking = bevolking[bevolking['Woonplaats'].str.find('Provincie') == -1]
bevolking = bevolking[bevolking['Woonplaats'].str.find('Arrondissement') == -1]

Schrijf een functie die op basis van een NIS code de juiste provincie retourneert. De logica is als volgt:

<table border="1">
    <tr>
        <th>Patroon</th>
        <th>Provincie</th>
    </tr>
    <tr>
        <td>1xx</td>
        <td>Antwerp</td>
    </tr>
    <tr>
        <td>21x</td>
        <td>Brussel</td>
    </tr>
    <tr>
        <td>23x</td>
        <td>Vlaams-Brabant</td>
    </tr>
    <tr>
        <td>24x</td>
        <td>Vlaams-Brabant</td>
    </tr>
    <tr>
        <td>25x</td>
        <td>Waals-Brabant</td>
    </tr>
    <tr>
        <td>3xx</td>
        <td>West-Vlaanderen</td>
    </tr>
    <tr>
        <td>4xx</td>
        <td>Oost-Vlaanderen</td>
    </tr>
    <tr>
        <td>5xx</td>
        <td>Henegouwen</td>
    </tr>
    <tr>
        <td>6xx</td>
        <td>Luik</td>
    </tr>
    <tr>
        <td>7xx</td>
        <td>Limburg</td>
    </tr>
    <tr>
        <td>8xx</td>
        <td>Luxemburg</td>
    </tr>
    <tr>
        <td>9xx</td>
        <td>Namen</td>
    </tr>
</table>

Bron: https://nl.wikipedia.org/wiki/NIS-code

In [15]:
def provincie_op_basis_van_nis(code):

    code = str(code)
    
    if code[0] == '1': return 'Antwerp'
    if code[0] == '2'  and code[1] == '1': return 'Brussel'
    if code[0] == '2'  and code[1] == '3': return 'Vlaams-Brabant'
    if code[0] == '2'  and code[1] == '4': return 'Vlaams-Brabant'
    if code[0] == '2'  and code[1] == '5': return 'Waals-Brabant'
    if code[0] == '3': return 'West-Vlaanderen'
    if code[0] == '4': return 'Oost-Vlaanderen'
    if code[0] == '5': return 'Henegouwen'
    if code[0] == '6': return 'Luik'
    if code[0] == '7': return 'Limburg'
    if code[0] == '8': return 'Luxemburg'
    if code[0] == '9': return 'Namen'

    return 'Ongekende provincie'

Gebruik deze functie om provincie af te leiden op basis van de NIS Code.<br />
Tip: Gebruik de apply functie https://www.digitalocean.com/community/tutorials/pandas-dataframe-apply-examples

In [16]:
bevolking['Provincie'] = bevolking['NIS code'].apply(provincie_op_basis_van_nis)

<h2>Data analyse - Eén ster oefeningen</h2>

Hoeveel verschillende NIS-codes bestaan er?

In [ ]:
print(bevolking.groupby('NIS code')['NIS code'].count())
len(bevolking.groupby('NIS code')['NIS code'].count())

Hoeveel jaren zijn er beschreven in de dataset?

In [ ]:
print(bevolking.groupby('Jaartal')['Jaartal'].count())
len(bevolking.groupby('Jaartal')['Jaartal'].count())

Wat is het aantal inwoners van heel België in het jaar 2023?

In [ ]:
bevolking_2023 = bevolking[bevolking['Jaartal'] == 2023]
bevolking_2023['Aantal'].sum()

<h2>Data analyse - Twee ster oefeningen</h2>

Wat is de evolutie van het aantal inwoners van heel België over de jaren heen?

In [ ]:
evolutie_bevolking = bevolking.groupby('Jaartal')['Aantal'].sum()

plt.bar(evolutie_bevolking.index.tolist(), evolutie_bevolking.tolist())
plt.show()

Wat is het totaal aantal inwoners per provincie, per jaar?

In [ ]:
pivot = bevolking.pivot_table(index='Provincie', columns='Jaartal', values='Aantal', aggfunc='sum')
pivot.head(10)

<h2>Data analyse - Drie ster oefeningen</h2>

Wat is het hoogste aantal inwoners dat elke provincie ooit heeft behaald?

In [ ]:
inwoners_per_provincie_per_jaar = pd.DataFrame(bevolking.groupby(['Provincie','Jaartal'])['Aantal'].sum())
inwoners_per_provincie_per_jaar.groupby('Provincie')['Aantal'].max()

In welke provincies wonen er meer mannen dan vrouwen?

In [ ]:
vrouwen = bevolking[bevolking['Geslacht'] == 'Vrouwen']
mannen = bevolking[bevolking['Geslacht'] == 'Mannen']

vrouwen_per_provincie = pd.DataFrame(vrouwen.groupby(['Provincie'])['Aantal'].sum())
mannen_per_provincie = pd.DataFrame(mannen.groupby(['Provincie'])['Aantal'].sum())

vergelijking = mannen_per_provincie.merge(vrouwen_per_provincie, on='Provincie', how='inner')
vergelijking[vergelijking['Aantal_x'] < vergelijking['Aantal_y']]